# Estudo de Preços e Combustíveis

Usando Cassandra, Python e Jupyter Notebook

### Declaração de classses de dados

In [74]:
class Vendas():
    def __init__(self, regiao, estado, municipio, revenda, 
                 produto, dt_coleta, valor_compra, 
                 valor_venda, unidade_medida, bandeira):
        self.regiao = regiao
        self.estado = estado
        self.municipio = municipio
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        self.mes = int(dt_coleta[3:5])
        self.ano = int(dt_coleta[8:10])

        self.valor_compra = None
        if (valor_compra != ''):
            self.valor_compra = float(valor_compra.replace(',', '.'))

        self.valor_venda = None
        if (valor_venda != ''):
            self.valor_venda = float(valor_venda.replace(',', '.'))
        
        self.unidade_medida = unidade_medida
        self.bandeira = bandeira

    def get_regiao(self):
        return self.regiao

    def get_estado(self):
        return self.estado

    def get_municipio(self):
        return self.municipio

    def get_revenda(self):
        return self.revenda

    def get_produto(self):
        return self.produto    
        
    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_mes(self):
        return self.mes
    
    def get_ano(self):
        return self.ano
    
    def get_valor_compra(self):
        return self.valor_compra
    
    def get_valor_venda(self):
        return self.valor_venda
    
    def get_unidade_medida(self):
        return self.unidade_medida        
    
    def get_bandeira(self):
        return self.bandeira     

In [80]:
class Vendas_Valor():
    def __init__(self, municipio, bandeira, revenda, produto, dt_coleta, valor_venda):
        self.municipio = municipio
        self.bandeira = bandeira
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        self.valor_venda = valor_venda 
            
    def get_municipio(self):
        return self.municipio
    
    def get_bandeira(self):
        return self.bandeira
    
    def get_revenda(self):
        return self.revenda    
    
    def get_produto(self):
        return self.produto    

    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_valor_venda(self):
        return self.valor_venda    

In [52]:
class Dolares_Mes():
    def __init__(self, ano, mes, valor_compra):
        self.ano = ano
        self.mes = mes
        self.valor_compra = valor_compra

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_valor_compra(self):
        return self.valor_compra

In [53]:
class Combustivel_Mes():
    def __init__(self, ano, mes, produto):
        self.ano = ano
        self.mes = mes
        self.produto = produto
        self.qtd = qtd
        self.valor_venda = valor_venda        

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_produto(self):
        return self.produto
    
    def get_qtd(self):
        return self.qtd
    
    def get_valor_venda(self):
        return self.valor_venda

In [54]:
class Media_Mes():
    def __init__(self, ano, mes, diesel, diesel_s10, etanol, gasolina, gnv):
        self.ano = ano
        self.mes = mes
        self.dolar = dolar
        self.diesel = diesel
        self.diesel_s10 = diesel_s10        
        self.etanol = etanol
        self.gasolina = gasolina        
        self.gnv = gnv        

    def get_ano(self):
        return self.ano
    
    def get_mes(self):
        return self.mes
    
    def get_dolar(self):
        return self.dolar
    
    def get_diesel(self):
        return self.diesel
    
    def get_diesel_s10(self):
        return self.diesel_s10
    
    def get_etanol(self):
        return self.etanol
    
    def get_gasolina(self):
        return self.gasolina
    
    def get_gnv(self):
        return self.gnv    

### Declaração de conexões e carregadores de dados

In [55]:
def load_vendas_info_file(fname):
    vendas = []
    with open(fname, encoding="latin-1") as f:
        next(f)
        for line in f: 
            vendas.append(Vendas(*str(line).strip().split('\t')))
    f.close()
    return vendas

In [56]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

CASSANDRA_USER = 'cassandra'
CASSANDRA_PASSWD = 'password'

def create_connection(nodeIP, keyspaceName):
    """
    Function to create a connection object
    """

    client = Connection()
    client.connect(nodeIP) #Observar que o nodeIP é uma lista e não somente um único valor
    client.set_keyspace(keyspaceName)
    return client


class Connection:
    session = None
    keyspace = ""

    def connect(self, nodes):
        auth_provider = PlainTextAuthProvider(username=str(CASSANDRA_USER), password=str(CASSANDRA_PASSWD))
        cluster = Cluster(nodes, auth_provider=auth_provider)
        metadata = cluster.metadata
        self.session = cluster.connect()

    def get_session():
        return self.session

    def set_keyspace(self, keyspace):
        self.keyspace = keyspace

    def get_keyspace(self):
        return self.keyspace

    def close(self):
        self.session.cluster.shutdown()
        self.session.shutdown()

    def query_schema(self, table):
        table_keyspace = self.keyspace+"."+table
        query = " SELECT * FROM "+ table_keyspace +";"
        result_set = self.session.execute(query)
        return result_set

In [58]:
#Classe pai das conexões
class CommonCassandraDB():

    def __init__(self):
        self.cassandra_table_reference = None
        self.client = None

    def set_cassandra_client(self, nodeIP, keyspaceName):
        self.client = create_connection(nodeIP, keyspaceName)

    def close_connection(self):
        self.client.close()

### Declaração de classes com conexões

In [107]:
class VendasValorCassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'vendas_valor' #nome da Tabela do Cassandra
        self.set_cassandra_client(nodeIP, keyspace)
        
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = 'SELECT * FROM @$$$$$$$$$$$$@'.replace("@$$$$$$$$$$$$@", table)
        return self.client.session.execute(sql_query) 
    
    def get_mais_caros(self, municipio, produto, limite):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
            SELECT *
            FROM @TABELA
            WHERE municipio = '@MUNICIPIO'
            AND produto = '@PRODUTO'
            ORDER BY valor_venda DESC
            LIMIT @LIMITE
            """
        sql_query = sql_query.replace("@TABELA", table)
        sql_query = sql_query.replace("@MUNICIPIO", municipio)
        sql_query = sql_query.replace("@PRODUTO", produto)
        sql_query = sql_query.replace("@LIMITE", str(limite))
        
        return self.client.session.execute(sql_query)
    
    def get_mais_baratos(self, municipio, produto, limite):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
            SELECT *
            FROM @TABELA
            WHERE municipio = '@MUNICIPIO'
            AND produto = '@PRODUTO'
            ORDER BY valor_venda
            LIMIT @LIMITE
            """
        sql_query = sql_query.replace("@TABELA", table)
        sql_query = sql_query.replace("@MUNICIPIO", municipio)
        sql_query = sql_query.replace("@PRODUTO", produto)
        sql_query = sql_query.replace("@LIMITE", str(limite))
        
        return self.client.session.execute(sql_query)
 
    def show_row(self, row):
        print(row.get_municipio()) 
        print(row.get_bandeira())
        print(row.get_revenda()) 
        print(row.get_produto())
        print(row.get_dt_coleta())
        print(row.get_valor_venda())

    def insert(self, venda_valor):
        try:
            table = self.client.get_keyspace() + "." + self.cassandra_table_reference
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                municipio, 
                bandeira, 
                revenda, 
                produto, 
                dt_coleta, 
                valor_venda
            )
            VALUES (?,?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    venda_valor.get_municipio(),
                    venda_valor.get_bandeira(),
                    venda_valor.get_revenda(),
                    venda_valor.get_produto(),
                    venda_valor.get_dt_coleta(),
                    venda_valor.get_valor_venda()              
                    ))
                )
            return True
        except:
            raise
        return False

In [82]:
class CombustivelMesCassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'combustivel_mes' #nome da Tabela do Cassandra
        self.set_cassandra_client(nodeIP, keyspace)
        
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = 'SELECT * FROM @$$$$$$$$$$$$@'.replace("@$$$$$$$$$$$$@", table)
        return self.client.session.execute(sql_query) 

    def show_row(self, row):
        print(row.get_municipio()) 
        print(row.get_bandeira())
        print(row.get_produto()) 
        print(row.get_qtd())
        print(row.get_valor_venda())

    def insert(self, venda_valor):
        try:
            table = self.client.get_keyspace() + "." + self.cassandra_table_reference
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                ano, 
                mes, 
                produto, 
                qtd, 
                valor_venda
            )
            VALUES (?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    venda_valor.get_ano(),
                    venda_valor.get_mes(),
                    venda_valor.get_produto(),
                    venda_valor.get_qtd(),
                    venda_valor.get_valor_venda()              
                    ))
                )
            return True
        except:
            raise
        return False

In [83]:
class VendasCassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'vendas' #nome da Tabela do Cassandra
        self.set_cassandra_client(nodeIP, keyspace)
        
        self.vendasValorDAO = VendasValorCassandra(nodeIP, keyspace)
        
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = 'SELECT * FROM @$$$$$$$$$$$$@'.replace("@$$$$$$$$$$$$@", table)
        query_result = self.client.session.execute(sql_query) 
        print(query_result)
        return query_result

    def show_row(self, row):
        print(row.get_regiao()) 
        print(row.get_estado())
        print(row.get_municipio()) 
        print(row.get_revenda())
        print(row.get_produto())
        print(row.get_dt_coleta())
        print(row.get_mes())
        print(row.get_ano())
        print(row.get_valor_compra())
        print(row.get_valor_venda())
        print(row.get_unidade_medida())
        print(row.get_bandeira())

    def insert(self, venda):
        try:
            table = self.client.get_keyspace() + "." + self.cassandra_table_reference
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                regiao, 
                estado, 
                municipio, 
                revenda, 
                produto, 
                dt_coleta, 
                mes,
                ano,
                valor_compra, 
                valor_venda, 
                unidade_medida, 
                bandeira
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    venda.get_regiao(),
                    venda.get_estado(),
                    venda.get_municipio(),
                    venda.get_revenda(),
                    venda.get_produto(),
                    venda.get_dt_coleta(),
                    venda.get_mes(),
                    venda.get_ano(),
                    venda.get_valor_compra(), 
                    venda.get_valor_venda(), 
                    venda.get_unidade_medida(), 
                    venda.get_bandeira()                
                    ))
                )
            
            # Adicionar valor para o consultor de topo de valores
            self.vendasValorDAO.insert(
                Vendas_Valor(
                    venda.get_municipio(), 
                    venda.get_bandeira(), 
                    venda.get_revenda(), 
                    venda.get_produto(), 
                    venda.get_dt_coleta(), 
                    venda.get_valor_venda()
                )
            )        
            
            return True
        except:
            raise
        return False

### Carregar arquivos originais

[Segundo Semestre de 2016](http://www.anp.gov.br/images/dadosabertos/precos/2016-2_CA.csv)  
[Primeiro Semestre de 2017](http://www.anp.gov.br/images/dadosabertos/precos/2017-1_CA.csv)  
[Segundo Semestre de 2017](http://www.anp.gov.br/images/dadosabertos/precos/2017-2_CA.csv)  
[Primeiro Semestre de 2018](http://www.anp.gov.br/images/dadosabertos/precos/2018-1_CA.csv)  

In [111]:
vendasDAO = VendasCassandra(['172.17.0.2'], 'gas_price')

#for file in ['2016-2_CA.csv', '2017-1_CA.csv', '2017-2_CA.csv', '2018-1_CA.csv']:
for file in ['teste.csv']:
    for venda in load_vendas_info_file(file):
        vendasDAO.insert(venda)

In [145]:
import pandas
from datetime import datetime

# Carregar arquivo
dolar_diario = pandas.read_csv('bcdata.sgs.10813.csv', sep=";")

# Converter data para datetime
converterData = lambda data: datetime.strptime(str(data), '%d/%m/%Y')
dolar_diario['data'] = dolar_diario['data'].apply(converterData)
dolar_diario['valor'] = dolar_diario['valor'].apply(lambda valor: float(valor.replace(',', '.')))

# Filtrar por data
dolar_diario = dolar_diario[dolar_diario['data'] >= datetime(2016, 7, 1)]
dolar_diario = dolar_diario[dolar_diario['data'] <= datetime(2018, 6, 30)]

# Separar mês e ano
dolar_diario['mes'] = dolar_diario['data'].apply(lambda data: data.month)
dolar_diario['ano'] = dolar_diario['data'].apply(lambda data: data.year)

# Visualizar
dolar_diario = dolar_diario.drop('data', 1)
print(dolar_diario.groupby(['ano', 'mes']).mean())

             valor
ano  mes          
2016 7    3.274967
     8    3.209057
     9    3.255771
     10   3.185245
     11   3.341430
     12   3.351668
2017 1    3.195986
     2    3.103589
     3    3.127330
     4    3.135572
     5    3.208705
     6    3.294767
     7    3.205538
     8    3.150313
     9    3.134190
     10   3.190638
     11   3.258735
     12   3.291310
2018 1    3.209986
     2    3.240906
     3    3.278614
     4    3.406900
     5    3.635462
     6    3.772571


In [163]:
dolares = []
for index, dolar in dolar_diario.iterrows():
    dolares = Dolares_Mes(dolar['ano'], dolar['mes'], dolar['valor'])

In [ ]:
dolaresMesDAO = DolaresMesCassandra(['172.17.0.2'], 'gas_price')

dolaresMesDAO.insert

### Geração de dados para análise

In [85]:
vendasDAO = VendasCassandra(['172.17.0.2'], 'gas_price')

In [86]:
for venda in vendasDAO.get_full_table():
    print(venda)

Row(regiao='CO', estado='DF', municipio='BRASILIA', revenda='AMORIM COMERCIO DE DERIVADOS DE PETROLEO LTDA', produto='DIESEL', bandeira='IPIRANGA', dt_coleta='26/07/2016', ano=16, mes=7, unidade_medida='R$ / litro', valor_compra=None, valor_venda=3.2699999809265137)
Row(regiao='CO', estado='DF', municipio='BRASILIA', revenda='AMORIM E AMORIM EMPREENDIMENTOS COMERCIAIS', produto='DIESEL', bandeira='IPIRANGA', dt_coleta='12/07/2016', ano=16, mes=7, unidade_medida='R$ / litro', valor_compra=2.8171000480651855, valor_venda=3.1700000762939453)
Row(regiao='CO', estado='DF', municipio='BRASILIA', revenda='AMORIM E AMORIM EMPREENDIMENTOS COMERCIAIS', produto='DIESEL', bandeira='IPIRANGA', dt_coleta='26/07/2016', ano=16, mes=7, unidade_medida='R$ / litro', valor_compra=2.8239998817443848, valor_venda=3.1700000762939453)
Row(regiao='CO', estado='DF', municipio='BRASILIA', revenda='AUTO POSTO ALMENARA LTDA.', produto='DIESEL', bandeira='RAIZEN', dt_coleta='03/08/2016', ano=16, mes=8, unidade_medi

### Análises

In [109]:
VendasValorDAO = VendasValorCassandra(['172.17.0.2'], 'gas_price')

In [105]:
for venda in VendasValorDAO.get_mais_caros('SAO JOSE DO RIO PRETO', 'ETANOL', 10):
    print(venda)

Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.8989999294281006, bandeira='PETROBRAS DISTRIBUIDORA S.A.', revenda='AUTO POSTO PEROLA - RIO PRETO LTDA.', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.8889999389648438, bandeira='IPIRANGA', revenda='AUTO POSTO REX RIO PRETO LTDA', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.878999948501587, bandeira='IPIRANGA', revenda='AUTO POSTO IMPERIAL RIO PRETO LTDA.', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.878999948501587, bandeira='IPIRANGA', revenda='AUTO POSTO GENERAL LTDA', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.8589999675750732, bandeira='IPIRANGA', revenda='PORCINI & PORCINI LTDA', dt_coleta='01/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.8589999675750732, bandeira='IPIRANGA'

In [110]:
for venda in VendasValorDAO.get_mais_baratos('SAO JOSE DO RIO PRETO', 'ETANOL', 10):
    print(venda)
    
for venda in VendasValorDAO.get_mais_baratos('SAO JOSE DO RIO PRETO', 'ETANOL', 10):
    print(venda)

Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.6489999294281006, bandeira='BRANCA', revenda='AUTO POSTO RIO PIEDADE LTDA', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.6700000762939453, bandeira='BRANCA', revenda='AUTO POSTO VILA AZUL - RIO PRETO LTDA.', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.678999900817871, bandeira='BRANCA', revenda='AUTO POSTO FORTUNATO LTDA', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.678999900817871, bandeira='BRANCA', revenda='R PORCINI & CIA LTDA', dt_coleta='01/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.678999900817871, bandeira='IPIRANGA', revenda='AUTO POSTO PARATI RIO PRETO LTDA.', dt_coleta='09/11/2016')
Row(municipio='SAO JOSE DO RIO PRETO', produto='ETANOL', valor_venda=2.678999900817871, bandeira='PETROBRAS DISTRIBUIDORA S.A.', revenda='